# Introduction

This notebooks shows how to use Felis Cloud to benchmark the capabilities of Boson 4, Alice & Bob's first quantum chip on the cloud!

We will run experiments to characterize the performances of the cat-qubit operations. The user is given the following operations:

- Preparation of $|0\rangle$
- Preparation of $|1\rangle$
- Preparation of $|+\rangle$
- Measurement of Pauli operator $X$
- Measurement of Pauli operator $Z$
- Idling during an arbitrary time $t$
- $X(\pi)-$gate (purely virtual gate for cat qubits)
- $Z(\pi)-$gate

<img src="img/cat-blochsphere-01.png" style="width:50%">
<center> FIG 1: Definition of the Bloch sphere of the cat-qubit </center>

All these operations can be realized for various cat sizes, ranging from 4 to 10 photons.

The most important property of cat qubits is that bit-flip errors are exponentially suppressed with the number of photons. We verify this property in part 3 of the notebook, measuring bit-flip times above 1 second.

Interestingly, the physics behind the exponential suppression of bit-flip also allows to have exponentially precise state preparation and measurement (SPAM) of the observable Z. We will verify this property in part 4.

> Since cat qubits make so few bit-flips, measuring them can be difficult. As a consequence, we may need to average millions of times, to observe even a single bit-flip. So brace yourselves in such situations, because the measurements can last a bit.

<img src="img/cat-blochsphere-bf-pf.png" style="width:40%">
<center> FIG 2: A bit-flip correspond to a flip in between the south and north pole of the Bloch sphere. Bit-flips are exponentially suppressed in cat qubits. On the contrary, phase-flips (flip from one side of the equator to the other) increase linearly and must be corrected by an error correcting code, like the repetition code. </center>

Then, in part 5, we measure the phase-flip time, and verify that it increases only linearly with the number of photons.

Finally, in parts 6 and 7, we characterize the Z-gate. In particular, we verify that it is bias-preserving.

> The bit-flip time of cat qubits is many orders of magnitude larger than their phase-flip time. This property is coined **noise-bias**, and is what makes Quantum Error Correction of cat qubits so efficient. A crucial property of any operations performed on cat qubits is therefore that the operation preserve this noise bias. In this case, we say that the operation is **bias-preserving**. In part 7, we indeed verify that the Z-gate is bias-preserving.

> The fact that the operations must be bias-preserving also forbids to perform $X(\pi/2)$ and $Y(\pi/2)$ gates, since these convert a phase-flip into a bit-flip. That's why our physical gate-set is pretty limited and not universal at the physical level. However, reassure yourself, we can make a universal gate set at the logical level out of a non-universal gate set at the physical level. See our logical qubit emulator for more details!

# Helper functions

We start with a few import and helper functions that you can run blindly for the moment.

## Import and some functions

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
from qiskit import QuantumCircuit, transpile
from qiskit.providers import BackendV2
from time import sleep
import traceback

# set default plot parameters
plt.rcParams.update(
    {
        "figure.facecolor": "white",
        "axes.facecolor": "white",
        "font.family": "serif",
        "font.size": 16,
        "figure.dpi": 72.0,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "xtick.major.size": 5.0,
        "xtick.minor.size": 2.5,
        "ytick.major.size": 5.0,
        "ytick.minor.size": 2.5,
        "xtick.minor.visible": True,
        "ytick.minor.visible": True,
        "axes.grid": True,
        "axes.titlesize": "larger",
        "axes.labelsize": "larger",
        "legend.fontsize": "larger",
        "grid.color": "gray",
        "grid.linestyle": "--",
        "grid.alpha": 0.3,
        "lines.linewidth": 1.0,
        "figure.figsize": (16, 8),
    }
)


def build_bit_flip_circuit(
    delay_duration_s: float, add_z: int = 0, add_x: bool = False
) -> QuantumCircuit:
    circ = QuantumCircuit(1, 1)
    if add_x:
        circ.initialize("1", 0)
    else:
        circ.initialize("0", 0)
    for _ in range(add_z):
        circ.z(0)
    if delay_duration_s > 0:
        circ.delay(delay_duration_s, 0, unit="s")
    circ.measure(0, 0)
    return circ


def build_phase_flip_circuit(
    delay_duration_s: float,
    cat_qubit_arch: bool,
    add_z: int = 0,
) -> QuantumCircuit:
    circ = QuantumCircuit(1, 1)
    if cat_qubit_arch:
        circ.initialize("+", 0)
    else:
        circ.h(0)
    for _ in range(add_z):
        circ.z(0)
    if delay_duration_s > 0:
        circ.delay(delay_duration_s, 0, unit="s")
    if cat_qubit_arch:
        circ.measure_x(0, 0)
    else:
        circ.h(0)
        circ.measure(0, 0)
    return circ


def mean_std_X(counts: dict) -> tuple[float, float]:
    """
    Computes mean value and standard deviation of measurement of observable Pauli X
    """
    nb_shots = counts.get("0", 0) + counts.get("1", 0)
    p = counts.get("1", 0) / nb_shots
    std = np.sqrt(p * (1 - p) / nb_shots)
    mean_X = 1 - 2 * p
    std_X = 2 * std
    return mean_X, std_X


def mean_std_Z(counts: dict) -> tuple[float, float]:
    """
    Computes mean value and standard deviation of measurement of observable Pauli Z
    """
    nb_shots = counts.get("0", 0) + counts.get("1", 0)
    p = counts.get("1", 0) / nb_shots
    std = np.sqrt(p * (1 - p) / nb_shots)
    std = np.maximum(std, 3 / nb_shots)
    mean_X = 2 * p - 1
    std_X = 2 * std
    return mean_X, std_X

## Access Alice & Bob hardware with Felis Cloud

The functions below use Felis, Alice & Bob's open-source Qiskit provider. 

We will use Felis to connect to a real Boson 4 chip, as well as to emulate cat qubits with performances similar to Boson 4's.

Running experiments on a real Boson 4 chip requires a Felis Cloud subscription available at [https://console.cloud.google.com/marketplace/product/cloud-prod-0/felis-cloud](https://console.cloud.google.com/marketplace/product/cloud-prod-0/felis-cloud).

In [ ]:
from qiskit_alice_bob_provider import AliceBobRemoteProvider, AliceBobLocalProvider

NB_PHOTONS_RANGE = [4.0, 6.0, 8.0, 10.0]


def create_boson_4_backend(average_nb_photons: float) -> BackendV2:
    ab_provider = AliceBobRemoteProvider(
        api_key="YOUR_API_KEY_HERE"
    )
    return ab_provider.get_backend(
        "QPU:1Q:BOSON_4A", average_nb_photons=average_nb_photons
    )


def create_theory_cat_backend(average_nb_photons: float) -> BackendV2:
    ab_provider = AliceBobLocalProvider()
    return ab_provider.get_backend(
        "EMU:6Q:PHYSICAL_CATS",
        average_nb_photons=average_nb_photons,
        kappa_1=2 * 3.14 * 2_260,
        kappa_2=2 * 3.14 * 250_000,
    )

# Bit-flip lifetime: P0 | Idle | Mz

To measure the bit-flip time, we play the following pulse sequence:

- Prepare $\langle Z \rangle_0 = \pm 1$
- Wait for some variable time $t$
- Measure $\langle Z \rangle_t$

We then fit $\langle Z \rangle_t = \pm e^{-t/T_Z}$, where the decay time $T_Z$ of the Pauli observable $Z$ is the bit-flip time.

Repeating this measurement for several number of photons $\bar{n}$, we expect to see an exponential increase of the bit-flip time:

$$T_Z \propto e^{\gamma\bar{n}}$$

Where the scaling factor $\gamma \leq 2$ in presence of pure dephasing of the oscillator.

> Note: in this section, we completely neglect SPAM errors. This will be justified by the measurements of the next section.

## Circuit

In [ ]:
cat_circuit = build_bit_flip_circuit(delay_duration_s=5e-6, add_x=True)
cat_circuit.draw("mpl")

## Helpers

Helper functions to measure $\langle Z \rangle_t$, fit $T_Z$ and plot the result.

In [ ]:
def measure_bitflip_time(nb_photons, time_range, shots):
    mean = np.zeros((time_range.size, 2), dtype=float)
    std = np.zeros_like(mean)
    backend = create_boson_4_backend(nb_photons)
    # backend = create_theory_cat_backend(nb_photons)
    for i, time in enumerate(time_range):
        for j, add_x in enumerate([True, False]):
            success = False
            while not success:
                try:
                    circuit = build_bit_flip_circuit(delay_duration_s=time, add_x=add_x)
                    transpiled_circ = transpile(circuit, backend)
                    counts = backend.run(transpiled_circ, shots=shots).result().get_counts()
                    m, s = mean_std_Z(counts)
                    mean[i, j] = m
                    std[i, j] = s
                    print(
                        f"nb_photons = {nb_photons} - time = {time} - add_x = "
                        f"{add_x} - <Z> = {mean[i, j]:.02e} +- {std[i, j]:.02e}"
                    )
                    success = True
                except Exception:
                    traceback.print_exc()
                    sleep(10)
    return mean, std


def fit_bitflip_time(time_range, mean, std):
    def fun(t, *p):
        T = p
        return np.exp(-t / T)

    x = time_range
    y = 0.5 * (mean[:, 0] - mean[:, 1])
    sigma = 0.5 * np.sqrt(std[:, 1] ** 2 + std[:, 0] ** 2)
    T_guess = -x[1] / np.log(y[1])
    p0 = [T_guess]
    popt, pcov = sp.optimize.curve_fit(
        fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True
    )
    T_z = popt[0]
    T_z_std = np.sqrt(pcov[0, 0])
    fit = fun(time_range, *popt)
    return fit, T_z, T_z_std


def plot_bitflip_time(time_range, mean, std, fit):
    plt.errorbar(
        time_range,
        mean[:, 0],
        yerr=std[:, 0],
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
        label="$P_0$",
    )
    plt.errorbar(
        time_range,
        -mean[:, 1],
        yerr=std[:, 1],
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
        label="$P_1$",
    )
    plt.plot(time_range, fit, label="Fit")
    plt.xlabel("Idle time (s)")
    plt.ylabel("abs(<Z>)")
    plt.legend()

## 4 photons

We start with the smaller number of photons, $\bar{n}=4$ photons. As can be seen below, the bit-flip time $T_Z=600$ us is already one order of magnitude large than the bare $T_1$ of the oscillator.

In [ ]:
nb_photons = 4.0
time_range = np.linspace(0, 3e-3, 11)
shots = 500
mean, std = measure_bitflip_time(nb_photons, time_range, shots)

In [ ]:
fit, T_z, T_z_std = fit_bitflip_time(time_range, mean, std)
print(f'{nb_photons} photons')
print(f'Bit-flip time: T_Z = ({T_z:.02e} +- {T_z_std:.02e}) s')

In [ ]:
T_z_4_photons = T_z
T_z_std_4_photons = T_z_std

In [ ]:
plot_bitflip_time(time_range, mean, std, fit)

## 7 photons

At $\bar{n}=7$ photons, the bit-flip time $T_Z$ increases to several tens of milliseconds. With such lifetimes, it becomes challenging to measure the complete decay of the exponential.

> Technical point: this is the main reason why we prepare both initial states $\langle Z \rangle_0 = -1$ and $\langle Z \rangle_0 = +1$. By taking the difference in between both curves, we remove any offset $\langle Z \rangle_{t\rightarrow\infty} \neq 0$ that may exist at long time and would lead to an incorrect estimation of $T_Z$.

In [ ]:
nb_photons = 7.0
time_range = np.linspace(0, 50e-3, 11)
shots = 500
mean, std = measure_bitflip_time(nb_photons, time_range, shots)

In [ ]:
fit, T_z, T_z_std = fit_bitflip_time(time_range, mean, std)
print(f'{nb_photons} photons')
print(f'Bit-flip time: T_Z = ({T_z:.02e} +- {T_z_std:.02e}) s')

In [ ]:
T_z_7_photons = T_z
T_z_std_7_photons = T_z_std

In [ ]:
plot_bitflip_time(time_range, mean, std, fit)

## 10 photons

At $\bar{n}=10$ photons, the bit-flip time $T_Z$ increases to several tens of seconds. This is the maximum we can measure with the current readout technique in a reasonable amount of time.

> In Réglade, Bocquet et al., arXiv:2307.06617 (2023), we show a quantum trajectory method which allows to measure the bit-flip time virtually up to arbitrarily large values.

In [ ]:
nb_photons = 10.0
time_range = np.linspace(0, 1.0, 11)
shots = 500
mean, std = measure_bitflip_time(nb_photons, time_range, shots)

In [ ]:
fit, T_z, T_z_std = fit_bitflip_time(time_range, mean, std)
print(f'{nb_photons} photons')
print(f'Bit-flip time: T_Z = ({T_z:.02e} +- {T_z_std:.02e}) s')

In [ ]:
T_z_10_photons = T_z
T_z_std_10_photons = T_z_std

In [ ]:
plot_bitflip_time(time_range, mean, std, fit)

## Scaling

Gathering the results of the previous sections, we plot the scaling of the bit-flip time $T_Z$ with $\bar{n}$ and fit it with an exponential function $\exp(\gamma \bar{n})$.

Thanks to this exponential scaling, bit-flips become quickly completely negligible. As a consequence, we can use a simple 1D repetition code tackling only phase-flips to realize a logical qubit. This leads to a massive reduction of the hardware overhead required for Quantum Error Correction (from x60 to x200).

In [ ]:
nb_photons_range = np.array([4.0, 7.0, 10.0])
T_z = np.array([T_z_4_photons, T_z_7_photons, T_z_10_photons])
T_z_std = np.array([T_z_std_4_photons, T_z_std_7_photons, T_z_std_10_photons])

In [ ]:
def fun(nbar, *p):
    T0, gamma = p
    return T0 * np.exp(gamma * nbar)
x = nb_photons_range
y = T_z
sigma = T_z_std
gamma_guess = np.log(y[0] / y[1]) / (x[0] - x[1])
T0_guess = y[0] / np.exp(gamma_guess * x[0])
p0 = [T0_guess, gamma_guess]
popt, pcov = sp.optimize.curve_fit(fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True)

In [ ]:
plt.errorbar(
    nb_photons_range,
    T_z,
    yerr=T_z_std,
    marker="x",
    markersize=8,
    capsize=5,
    linestyle="none",
)
plt.plot(nb_photons_range, fun(nb_photons_range, *popt))
plt.yscale("log")
plt.xlabel("Number of photons")
plt.ylabel("$T_Z$ (s)")
plt.title(
    f"$T_0=({popt[0]*1e6:.02f}\pm{np.sqrt(pcov[0,0])*1e6:.02f})$ us - "
    f"$\gamma=({popt[1]:.02f}\pm{np.sqrt(pcov[1,1]):.02f})$"
)

# SPAM: P0 | Mz and P1 | Mz

The physics behind the exponential increase of the bit-flip time $T_Z$ also allows to have exponentially precise state preparation and measurement (SPAM) of the observable Z.

We verify it by playing the following pulse sequence:

- Prepare state $|0\rangle$
- Measure $Z$

Ideally, we would only measure $0$'s. By counting the number of $1$'s we get, that is the number of times the Pauli observable $Z$ flips, we extract the SPAM error probability:

$$\epsilon_Z = \frac{N_1}{N_0 + N_1}$$

We can do the same experiment starting from state $|1\rangle$, in which case:

$$\epsilon_Z = \frac{N_0}{N_0 + N_1}$$

As we will see, the SPAM errors are suppressed exponentially with the number of photons:

$$\epsilon_Z \propto e^{-\gamma\bar{n}}$$

This is really nice, but also makes challenging to measure even a single flip of the observable $Z$ at large photon number. The way we proceed below is to start with 1000 shots, and increase the number of shots by a factor 10x until we get at least 100 flips of the observable. Eventually, we stop at 10_000_000 shots, which is the hard limit of the Cloud API.

## Circuit

In [ ]:
cat_circuit = build_bit_flip_circuit(delay_duration_s=0, add_x=True)
cat_circuit.draw("mpl")

## Scaling

In [ ]:
nb_photons_range = np.array([4.0, 7.0, 10.0])
N_pos = np.zeros((nb_photons_range.size, 2), dtype=int)
N_tot = np.zeros_like(N_pos, dtype=int)
for j, add_x in enumerate([True, False]):
    N = 1000
    for i, nb_photons in enumerate(nb_photons_range):
        backend = create_boson_4_backend(nb_photons)
        enough_counts = False
        reached_limit = False
        while (not enough_counts) and (not reached_limit):
            success = False
            while not success:
                try:
                    circuit = build_bit_flip_circuit(delay_duration_s=0, add_x=add_x)
                    transpiled_circuit = transpile(circuit, backend)
                    counts = backend.run(transpiled_circuit, shots=N).result().get_counts()
                    success = True
                except Exception:
                    traceback.print_exc()
                    sleep(10)
            N_pos[i, j] += counts.get("0", 0) if add_x else counts.get("1", 0)
            N_tot[i, j] += counts.get("0", 0) + counts.get("1", 0)
            print(
                f"nb_photons = {nb_photons} - add_x = {add_x} - N_pos = "
                f"{N_pos[i, j]} - N_tot = {N_tot[i, j]}"
            )
            if N_pos[i, j] > 100:
                enough_counts = True
            else:
                N *= 10
            if N > 10_000_000:
                reached_limit = True
                N = 10_000_000

In [ ]:
mean = N_pos / N_tot
std = np.maximum(np.sqrt(mean * (1 - mean) / N_tot), 3 / N_tot)
for i in range(2):
    plt.errorbar(
        nb_photons_range,
        mean[:, i],
        yerr=std[:, i],
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
    )
plt.gca().set_prop_cycle(None)
for i in range(2):

    def fun(nbar, *p):
        A, gamma = p
        return A * np.exp(-gamma * nbar)

    x = nb_photons_range
    y = mean[:, i]
    sigma = std[:, i]
    gamma_guess = np.log(y[0] / y[1]) / (x[1] - x[0])
    A_guess = y[0] / np.exp(-gamma_guess * x[0])
    p0 = [A_guess, gamma_guess]
    popt, pcov = sp.optimize.curve_fit(
        fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True
    )
    plt.plot(
        x,
        fun(x, *popt),
        label=f"$P_{[1, 0][i]}$: $\gamma={popt[1]:.02f} \pm {np.sqrt(pcov[1,1]):.02f}$",
    )
plt.yscale("log")
plt.legend()
plt.xlabel("Number of photons")
plt.ylabel("$\epsilon_Z(SPAM)$")

# Phase-flip lifetime: P+ | Idle | Mx

To measure the phase-flip time, we play the following pulse sequence:

- Prepare $\langle X \rangle_0 = 1$
- Wait for some variable time $t$
- Measure $\langle X \rangle_t$

We then fit $\langle X \rangle_t = \langle X \rangle_0 e^{-t/T_X}$, where the decay time $T_X$ of the Pauli observable $X$ is the phase-flip time.

> Unlike for the observable $Z$, SPAM errors of the observable $X$ are sizeable. They are even quite large in this chip. This is owing to a not great $T_1\approx10$ us of the oscillator, and not-best-in-class two-photon dissipation rate $\kappa_2\approx 2\pi\times200$ kHz. As a consequence, we will keep $\langle X \rangle_0$ as a free fit parameter. The SPAM error probability is then given by: $\epsilon_X = 0.5\times\left(1 - \langle X \rangle_0\right)$.

Repeating this measurement for several number of photons $\bar{n}$, we expect to see a linear decrease of the phase-flip time:

$$T_X = \frac{T_1}{2\bar{n}}$$

Where $T_1$ is the energy relaxation time of the oscillator (plus some corrections due to finite temperature and non-linear Purcell rates).

## Circuit

In [ ]:
cat_circuit = build_phase_flip_circuit(
    delay_duration_s=5e-6, cat_qubit_arch=True
)
cat_circuit.draw("mpl", cregbundle=False)

## Helpers

Helper functions to measure $\langle X \rangle_t$, fit $T_X$ and plot the result.

In [ ]:
def measure_phaseflip_time(nb_photons, time_range, shots):
    mean = np.zeros_like(time_range, dtype=float)
    std = np.zeros_like(mean)
    backend = create_boson_4_backend(nb_photons)
    # backend = create_theory_cat_backend(nb_photons)
    for i, time in enumerate(time_range):
        success = False
        while not success:
            try:
                circuit = build_phase_flip_circuit(
                    delay_duration_s=time, cat_qubit_arch=True
                )
                transpiled_circuit = transpile(circuit, backend)
                counts = backend.run(transpiled_circuit, shots=shots).result().get_counts()
                m, s = mean_std_X(counts)
                mean[i] = m
                std[i] = s
                print(
                    f"nb_photons = {nb_photons} - time = {time} - <X> = "
                    f"{mean[i]:.02e} +- {std[i]:.02e}"
                )
                success = True
            except Exception:
                traceback.print_exc()
                sleep(10)
    return mean, std


def fit_phaseflip_time(time_range, mean, std):
    def fun(t, *p):
        C_spam, T, offset = p
        return C_spam * np.exp(-t / T) + offset

    x = time_range
    y = mean
    sigma = std
    offset_guess = y[-1]
    C_spam_guess = y[0] - offset_guess
    T_guess = -x[1] / np.log((y[1] - offset_guess) / C_spam_guess)
    p0 = [C_spam_guess, T_guess, offset_guess]
    popt, pcov = sp.optimize.curve_fit(
        fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True
    )
    eps_x_spam = 0.5 * (1 - popt[0])
    eps_x_spam_std = 0.5 * np.sqrt(pcov[0, 0])
    T_x = popt[1]
    T_x_std = np.sqrt(pcov[1, 1])
    fit = fun(time_range, *popt)
    return fit, T_x, T_x_std, eps_x_spam, eps_x_spam_std


def plot_phaseflip_time(time_range, mean, std, fit):
    plt.errorbar(
        time_range * 1e6,
        mean,
        yerr=std,
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
    )
    plt.plot(time_range * 1e6, fit)
    plt.xlabel("Idle time (us)")
    plt.ylabel("<X>")

## 4 photons

We start with the smaller number of photons, $\bar{n}=4$ photons. Given the large SPAM errors on this chip, we must average heavily: 100_000 shots.

In [ ]:
nb_photons = 4.0
time_range = np.linspace(0, 5e-6, 21)
shots = 100_000
mean, std = measure_phaseflip_time(nb_photons, time_range, shots)

In [ ]:
fit, T_x, T_x_std, eps_x_spam, eps_x_spam_std = fit_phaseflip_time(time_range, mean, std)
print(f'{nb_photons} photons')
print(f'Phase-flip time: T_X = ({T_x*1e6:.02f} +- {T_x_std*1e6:.02f}) us')
print(f'SPAM: epsilon_X = ({eps_x_spam*100:.01f} +- {eps_x_spam_std*100:.01f}) %')

In [ ]:
T_x_4_photons = T_x
T_x_std_4_photons = T_x_std
eps_x_spam_4_photons = eps_x_spam
eps_x_spam_std_4_photons = eps_x_spam_std

In [ ]:
plot_phaseflip_time(time_range, mean, std, fit)

## 7 photons

In [ ]:
nb_photons = 7.0
time_range = np.linspace(0, 2.5e-6, 21)
shots = 100_000
mean, std = measure_phaseflip_time(nb_photons, time_range, shots)

In [ ]:
fit, T_x, T_x_std, eps_x_spam, eps_x_spam_std = fit_phaseflip_time(
    time_range, mean, std
)
print(f"{nb_photons} photons")
print(f"Phase-flip time: T_X = ({T_x*1e6:.02f} +- {T_x_std*1e6:.02f}) us")
print(f"SPAM: epsilon_X = ({eps_x_spam*100:.01f} +- {eps_x_spam_std*100:.01f}) %")

In [ ]:
T_x_7_photons = T_x
T_x_std_7_photons = T_x_std
eps_x_spam_7_photons = eps_x_spam
eps_x_spam_std_7_photons = eps_x_spam_std

In [ ]:
plot_phaseflip_time(time_range, mean, std, fit)

## 10 photons

SPAM errors increase with photon number and we must increase the averaging to have acceptable error bars: 200_000 shots.

In [ ]:
nb_photons = 10.0
time_range = np.linspace(0, 1.8e-6, 21)
shots = 200_000
mean, std = measure_phaseflip_time(nb_photons, time_range, shots)

In [ ]:
fit, T_x, T_x_std, eps_x_spam, eps_x_spam_std = fit_phaseflip_time(
    time_range, mean, std
)
print(f"{nb_photons} photons")
print(f"Phase-flip time: T_X = ({T_x*1e6:.02f} +- {T_x_std*1e6:.02f}) us")
print(f"SPAM: epsilon_X = ({eps_x_spam*100:.01f} +- {eps_x_spam_std*100:.01f}) %")

In [ ]:
T_x_10_photons = T_x
T_x_std_10_photons = T_x_std
eps_x_spam_10_photons = eps_x_spam
eps_x_spam_std_10_photons = eps_x_spam_std

In [ ]:
plot_phaseflip_time(time_range, mean, std, fit)

## Scaling

Gathering the results of the previous sections, we plot the linear scaling of the phase-flip time $T_X$ with $\bar{n}$ and fit it to extract the effective $T_1$ of the oscillator.

In [ ]:
nb_photons_range = np.array([4.0, 7.0, 10.0])
T_x = np.array([T_x_4_photons, T_x_7_photons, T_x_10_photons])
T_x_std = np.array([T_x_std_4_photons, T_x_std_7_photons, T_x_std_10_photons])
Gamma_x = 1 / T_x
Gamma_x_std = T_x_std / T_x**2
eps_x_spam = np.array(
    [eps_x_spam_4_photons, eps_x_spam_7_photons, eps_x_spam_10_photons]
)
eps_x_spam_std = np.array(
    [eps_x_spam_std_4_photons, eps_x_spam_std_7_photons, eps_x_spam_std_10_photons]
)

In [ ]:
def fun(nbar, *p):
    T1 = p
    return 2 * nbar / T1
x = nb_photons_range
y = Gamma_x
sigma = Gamma_x_std
p0 = [1 / np.mean(y / (2 * x))]
popt, pcov = sp.optimize.curve_fit(fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True)

In [ ]:
plt.errorbar(
    nb_photons_range,
    Gamma_x * 1e-6,
    yerr=Gamma_x_std * 1e-6,
    marker="x",
    markersize=8,
    capsize=5,
    linestyle="none",
)
plt.plot(nb_photons_range, fun(nb_photons_range, *popt) * 1e-6)
plt.xlabel("Number of photons")
plt.ylabel("$1 / T_X$ $(us^{-1})$")
plt.title(f"$T_1=({popt[0]*1e6:.02f}\pm{np.sqrt(pcov[0,0])*1e6:.02f})$ us")

In [ ]:
plt.errorbar(
    nb_photons_range,
    eps_x_spam,
    yerr=eps_x_spam_std,
    marker="x",
    markersize=8,
    capsize=5,
    linestyle="none",
)
plt.xlabel("Number of photons")
plt.ylabel("$\epsilon_X(SPAM)$")

# Z-gate phase-flip: P+ | Z^n | Mx

To characterize the Z-gate, we play the following pulse sequence:

- Prepare $\langle X \rangle_0 = 1$
- Play a number $n$ of Z-gate
- Measure $\langle X \rangle_n$

We then fit $\langle X \rangle_n = \langle X \rangle_0 (-C_X)^n$, where $C_X$ is the loss of contrast of the Pauli observable $X$ per Z-gate. The phase-flip error probability per Z-gate is then given by:

$$\epsilon_X = \frac{1 - C_X}{2}$$

Note that for the same reasons that SPAM errors of the observable $X$ are not good, phase-flip errors per Z-gate are not expected to be good either on this chip.

## Circuit

In [ ]:
cat_circuit = build_phase_flip_circuit(delay_duration_s=0, cat_qubit_arch=True, add_z=3)
cat_circuit.draw('mpl')

## Helpers

Helper functions to measure $\langle X \rangle_n$, fit $\epsilon_X$ and plot the result.

In [ ]:
def measure_phaseflip_Zgate(nb_photons, nb_gates_range, shots):
    mean = np.zeros_like(nb_gates_range, dtype=float)
    std = np.zeros_like(mean)
    backend = create_boson_4_backend(nb_photons)
    # backend = create_theory_cat_backend(nb_photons)
    for i, nb_gates in enumerate(nb_gates_range):
        success = False
        while not success:
            try:
                circuit = build_phase_flip_circuit(
                    delay_duration_s=0, cat_qubit_arch=True, add_z=nb_gates
                )
                transpiled_circuit = transpile(circuit, backend)
                counts = backend.run(transpiled_circuit, shots=shots).result().get_counts()
                m, s = mean_std_X(counts)
                mean[i] = m
                std[i] = s
                print(
                    f"nb_photons = {nb_photons} - nb_gates = {nb_gates} - <X> = "
                    f"{mean[i]:.02e} +- {std[i]:.02e}"
                )
                success = True
            except Exception:
                traceback.print_exc()
                sleep(10)
    return mean, std


def fit_phaseflip_Zgate(nb_gates_range, mean, std):
    def fun(n, *p):
        C_spam, C_Zgate, offset = p
        return C_spam * (-C_Zgate) ** n + offset

    x = nb_gates_range
    y = mean
    sigma = std
    offset_guess = y[-1]
    C_spam_guess = y[0] - offset_guess
    C_Zgate_guess = -(y[1] - offset_guess) / C_spam_guess
    p0 = [C_spam_guess, C_Zgate_guess, offset_guess]
    popt, pcov = sp.optimize.curve_fit(
        fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True
    )
    eps_x_spam = 0.5 * (1 - popt[0])
    eps_x_spam_std = 0.5 * np.sqrt(pcov[0, 0])
    eps_x_Zgate = 0.5 * (1 - popt[1])
    eps_x_Zgate_std = 0.5 * np.sqrt(pcov[1, 1])
    fit = fun(nb_gates_range, *popt)
    return fit, eps_x_Zgate, eps_x_Zgate_std, eps_x_spam, eps_x_spam_std


def plot_phaseflip_Zgate(nb_gates_range, mean, std, fit):
    plt.errorbar(
        nb_gates_range,
        mean,
        yerr=std,
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
    )
    plt.plot(nb_gates_range[::2], fit[::2])
    plt.plot(nb_gates_range[1::2], fit[1::2])
    plt.xlabel("Number of gates")
    plt.ylabel("<X>")

## 4 photons

We start with the smaller number of photons, $\bar{n}=4$ photons. Given the large SPAM errors on this chip, we must average heavily: 100_000 shots.

In [ ]:
nb_photons = 4.0
nb_gates_range = np.arange(20, dtype=int)
shots = 100_000
mean, std = measure_phaseflip_Zgate(nb_photons, nb_gates_range, shots)

In [ ]:
fit, eps_x_Zgate, eps_x_Zgate_std, eps_x_spam, eps_x_spam_std = fit_phaseflip_Zgate(
    nb_gates_range, mean, std
)
print(f"{nb_photons} photons")
print(f"Z-gate: epsilon_X = ({eps_x_Zgate*100:.01f} +- {eps_x_Zgate_std*100:.01f}) %")
print(f"SPAM: epsilon_X = ({eps_x_spam*100:.01f} +- {eps_x_spam_std*100:.01f}) %")

In [ ]:
eps_x_Zgate_4_photons = eps_x_Zgate
eps_x_Zgate_std_4_photons = eps_x_Zgate_std
eps_x_spam_4_photons = eps_x_spam
eps_x_spam_std_4_photons = eps_x_spam_std

In [ ]:
plot_phaseflip_Zgate(nb_gates_range, mean, std, fit)

## 7 photons

SPAM and Z-gate errors increase with photon number and we must increase the averaging to have acceptable error bars: 200_000 shots.

In [ ]:
nb_photons = 7.0
nb_gates_range = np.arange(10, dtype=int)
shots = 200_000
mean, std = measure_phaseflip_Zgate(nb_photons, nb_gates_range, shots)

In [ ]:
fit, eps_x_Zgate, eps_x_Zgate_std, eps_x_spam, eps_x_spam_std = fit_phaseflip_Zgate(
    nb_gates_range, mean, std
)
print(f"{nb_photons} photons")
print(f"Z-gate: epsilon_X = ({eps_x_Zgate*100:.01f} +- {eps_x_Zgate_std*100:.01f}) %")
print(f"SPAM: epsilon_X = ({eps_x_spam*100:.01f} +- {eps_x_spam_std*100:.01f}) %")

In [ ]:
eps_x_Zgate_7_photons = eps_x_Zgate
eps_x_Zgate_std_7_photons = eps_x_Zgate_std
eps_x_spam_7_photons = eps_x_spam
eps_x_spam_std_7_photons = eps_x_spam_std

In [ ]:
plot_phaseflip_Zgate(nb_gates_range, mean, std, fit)

## 10 photons

SPAM and Z-gate errors increase with photon number and we must increase again the averaging to have acceptable error bars: 400_000 shots.

In [ ]:
nb_photons = 10.0
nb_gates_range = np.arange(10, dtype=int)
shots = 400_000
mean, std = measure_phaseflip_Zgate(nb_photons, nb_gates_range, shots)

In [ ]:
fit, eps_x_Zgate, eps_x_Zgate_std, eps_x_spam, eps_x_spam_std = fit_phaseflip_Zgate(
    nb_gates_range, mean, std
)
print(f"{nb_photons} photons")
print(f"Z-gate: epsilon_X = ({eps_x_Zgate*100:.01f} +- {eps_x_Zgate_std*100:.01f}) %")
print(f"SPAM: epsilon_X = ({eps_x_spam*100:.01f} +- {eps_x_spam_std*100:.01f}) %")

In [ ]:
eps_x_Zgate_10_photons = eps_x_Zgate
eps_x_Zgate_std_10_photons = eps_x_Zgate_std
eps_x_spam_10_photons = eps_x_spam
eps_x_spam_std_10_photons = eps_x_spam_std

In [ ]:
plot_phaseflip_Zgate(nb_gates_range, mean, std, fit)

## Scaling

In [ ]:
nb_photons_range = np.array([4.0, 7.0, 10.0])
eps_x_Zgate = np.array(
    [eps_x_Zgate_4_photons, eps_x_Zgate_7_photons, eps_x_Zgate_10_photons]
)
eps_x_Zgate_std = np.array(
    [eps_x_Zgate_std_4_photons, eps_x_Zgate_std_7_photons, eps_x_Zgate_std_10_photons]
)
eps_x_spam = np.array(
    [eps_x_spam_4_photons, eps_x_spam_7_photons, eps_x_spam_10_photons]
)
eps_x_spam_std = np.array(
    [eps_x_spam_std_4_photons, eps_x_spam_std_7_photons, eps_x_spam_std_10_photons]
)

In [ ]:
plt.errorbar(
    nb_photons_range,
    eps_x_Zgate,
    yerr=eps_x_Zgate_std,
    marker="x",
    markersize=8,
    capsize=5,
    linestyle="none",
)
plt.xlabel("Number of photons")
plt.ylabel("$\epsilon_X(Z(\pi))$")

In [ ]:
plt.errorbar(
    nb_photons_range,
    eps_x_spam,
    yerr=eps_x_spam_std,
    marker="x",
    markersize=8,
    capsize=5,
    linestyle="none",
)
plt.xlabel("Number of photons")
plt.ylabel("$\epsilon_X(SPAM)$")

# Z-gate bit-flip: P0 | Z^n | Mz

A crucial property of cat-qubit gates is  that they should be **bias-preserving**. That is, bit-flips should remain exponentially suppressed during the gate. To characterize the bias-preservingness of the Z-gate, we play the following pulse sequence:

- Prepare $\langle Z \rangle_0 = \pm 1$
- Play a number $n$ of Z-gate
- Measure $\langle Z \rangle_n$

We then fit $\langle Z \rangle_n = \langle Z \rangle_0 C_Z^n$, where $C_Z$ is the loss of contrast of the Pauli observable $Z$ per Z-gate. The bit-flip error probability per Z-gate is then given by:

$$\epsilon_Z = \frac{1 - C_Z}{2}$$

The Z-gate is bias-preserving if $\epsilon_Z$ is suppressed exponentially with the number of photons:

$$\epsilon_Z \propto e^{-\gamma\bar{n}}$$

> Note that we can sometimes observe in this measurement a small bias towards state $|0\rangle$ or $|1\rangle$. This is due to calibrations drift and we're working at improving the stability.

> Also note that, owing to compilation inefficiencies of the first release of the Cloud API, we are limited to play only a few hundred gates (hopefully this limitation will soon be removed!). Therefore, we can only measure $\epsilon_Z$ with acceptable error bars up to $\bar{n}=7$.

## Circuit

In [ ]:
cat_circuit = build_bit_flip_circuit(delay_duration_s=0, add_x=True, add_z=3)
cat_circuit.draw('mpl')

## Helpers

In [ ]:
def measure_bitflip_Zgate(nb_photons, nb_gates_range, shots):
    mean = np.zeros((nb_gates_range.size, 2), dtype=float)
    std = np.zeros_like(mean)
    backend = create_boson_4_backend(nb_photons)
    # backend = create_theory_cat_backend(nb_photons)
    for i, nb_gates in enumerate(nb_gates_range):
        for j, add_x in enumerate([True, False]):
            success = False
            while not success:
                try:
                    circuit = build_bit_flip_circuit(
                        delay_duration_s=0, add_z=nb_gates, add_x=add_x
                    )
                    transpiled_circuit = transpile(circuit, backend)
                    counts = backend.run(transpiled_circuit, shots=shots).result().get_counts()
                    m, s = mean_std_Z(counts)
                    mean[i, j] = m
                    std[i, j] = s
                    print(
                        f"nb_photons = {nb_photons} - nb_gates = {nb_gates} - "
                        f"add_x = {add_x} - <Z> = {mean[i, j]:.02e} +- {std[i, j]:.02e}"
                    )
                    success = True
                except Exception:
                    traceback.print_exc()
                    sleep(10)
    return mean, std


def fit_bitflip_Zgate(nb_gates_range, mean, std):
    def fun(n, *p):
        C_Zpi = p
        return C_Zpi**n

    x = nb_gates_range
    y = 0.5 * (mean[:, 0] - mean[:, 1])
    sigma = 0.5 * np.sqrt(std[:, 1] ** 2 + std[:, 0] ** 2)
    C_Zpi_guess = y[1]
    p0 = [C_Zpi_guess]
    popt, pcov = sp.optimize.curve_fit(
        fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True
    )
    eps_z_Zgate = 0.5 * (1 - popt[0])
    eps_z_Zgate_std = 0.5 * np.sqrt(pcov[0, 0])
    fit = fun(nb_gates_range, *popt)
    return fit, eps_z_Zgate, eps_z_Zgate_std


def plot_bitflip_Zgate(nb_gates_range, mean, std, fit):
    plt.errorbar(
        nb_gates_range,
        mean[:, 0],
        yerr=std[:, 0],
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
        label="$P_0$",
    )
    plt.errorbar(
        nb_gates_range,
        -mean[:, 1],
        yerr=std[:, 1],
        marker="x",
        markersize=8,
        capsize=5,
        linestyle="none",
        label="$P_1$",
    )
    plt.plot(nb_gates_range, fit, label="Fit")
    plt.xlabel("Number of gates")
    plt.ylabel("abs(<Z>)")
    plt.legend()

## 4 photons

In [ ]:
nb_photons = 4.0
nb_gates_range = np.arange(0, 200, 20, dtype=int)
shots = 500
mean, std = measure_bitflip_Zgate(nb_photons, nb_gates_range, shots)

In [ ]:
fit, eps_z_Zgate, eps_z_Zgate_std = fit_bitflip_Zgate(nb_gates_range, mean, std)
print(f'{nb_photons} photons')
print(f'Z-gate: epsilon_Z = ({eps_z_Zgate:.02e} +- {eps_z_Zgate_std:.02e})')

In [ ]:
eps_z_Zgate_4_photons = eps_z_Zgate
eps_z_Zgate_std_4_photons = eps_z_Zgate_std

In [ ]:
plot_bitflip_Zgate(nb_gates_range, mean, std, fit)

## 5.5 photons

In [ ]:
nb_photons = 5.5
nb_gates_range = np.array([0, 100, 200])
shots = 500_000
mean, std = measure_bitflip_Zgate(nb_photons, nb_gates_range, shots)

In [ ]:
fit, eps_z_Zgate, eps_z_Zgate_std = fit_bitflip_Zgate(nb_gates_range, mean, std)
print(f'{nb_photons} photons')
print(f'Z-gate: epsilon_Z = ({eps_z_Zgate:.02e} +- {eps_z_Zgate_std:.02e})')

In [ ]:
eps_z_Zgate_5p5_photons = eps_z_Zgate
eps_z_Zgate_std_5p5_photons = eps_z_Zgate_std

In [ ]:
plot_bitflip_Zgate(nb_gates_range, mean, std, fit)

## 7 photons

In [ ]:
nb_photons = 7.0
nb_gates_range = np.array([0, 100, 200])
shots = 500_000
mean, std = measure_bitflip_Zgate(nb_photons, nb_gates_range, shots)

In [ ]:
fit, eps_z_Zgate, eps_z_Zgate_std = fit_bitflip_Zgate(nb_gates_range, mean, std)
print(f'{nb_photons} photons')
print(f'Z-gate: epsilon_Z = ({eps_z_Zgate:.02e} +- {eps_z_Zgate_std:.02e})')

In [ ]:
eps_z_Zgate_7_photons = eps_z_Zgate
eps_z_Zgate_std_7_photons = eps_z_Zgate_std

In [ ]:
plot_bitflip_Zgate(nb_gates_range, mean, std, fit)

## Scaling

Gathering the results of the previous sections, we plot the scaling of the bit-flip error probability per Z-gate $\epsilon_Z$ with $\bar{n}$ and fit it with an exponential function $\exp(\gamma \bar{n})$.

The scaling factor $\gamma$ fitted on $\epsilon_Z$ has almost that fitted on $T_Z$. That is, bit-flips during the Z-gate and idling are similar.

In [ ]:
nb_photons_range = np.array([4.0, 5.5, 7.0])
eps_z_Zgate = np.array(
    [eps_z_Zgate_4_photons, eps_z_Zgate_5p5_photons, eps_z_Zgate_7_photons]
)
eps_z_Zgate_std = np.array(
    [eps_z_Zgate_std_4_photons, eps_z_Zgate_std_5p5_photons, eps_z_Zgate_std_7_photons]
)

In [ ]:
def fun(nbar, *p):
    A, gamma = p
    return A * np.exp(gamma * nbar)


x = nb_photons_range
y = eps_z_Zgate
sigma = eps_z_Zgate_std
gamma_guess = np.log(y[0] / y[1]) / (x[0] - x[1])
A_guess = y[0] / np.exp(gamma_guess * x[0])
p0 = [A_guess, gamma_guess]
popt, pcov = sp.optimize.curve_fit(fun, x, y, p0=p0, sigma=sigma, absolute_sigma=True)

In [ ]:
plt.errorbar(
    nb_photons_range,
    eps_z_Zgate,
    yerr=eps_z_Zgate_std,
    marker="x",
    markersize=8,
    capsize=5,
    linestyle="none",
)
plt.plot(nb_photons_range, fun(nb_photons_range, *popt))
plt.yscale("log")
plt.xlabel("Number of photons")
plt.ylabel("$\epsilon_Z(Z(\pi))$")
plt.title(f"$\gamma=({popt[1]:.02f}\pm{np.sqrt(pcov[1,1]):.02f})$")